<a href="https://colab.research.google.com/github/Lukehsu1999/PoetBot/blob/main/Classify_Poems_by_Pretrained_Emotion_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Import pretrained emotionClassifier </h1> 

Distribution of data based on labels:  <br>

joy    6761 <br>
sadness    5797 <br>
anger    2709 <br>
fear    2373 <br>
love    1641 <br>
surprise     719 <br>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 5.6MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 1.1MB 31.8MB/s 
     |████████████████████████████████| 2.9MB 29.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f02abd9c9a3ad0a5c833ee48b5ad432d550015f84691bc25fa4802996acf8beb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertForSequenceClassification

emotionClassifier=BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/EmotionNLP/pretrainedModel", return_dict=True)


In [ ]:
def classify_emotion(sentence):

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  input = tokenizer(sentence, return_tensors="pt")
  emotionClassifier.eval()
  labels=torch.tensor([1]).unsqueeze(0)

  outputs = emotionClassifier(**input,labels=labels)
  loss = outputs.loss
  logits = outputs.logits
  logits=logits.detach().numpy()
  probs = tf.nn.softmax(logits)
  prediction = tf.math.argmax(probs, axis=1)

  # 0:"anger", 1:"fear", 2:"joy", 3:"love", 4:"sadness", 5:"surprise",
  prediction=tf.constant(prediction)

  #print(probs)
  probs=tf.constant(probs)
  prob_prediction=probs.numpy()[0][prediction]
  prob_prediction=tf.constant(prob_prediction)
  #print(prediction)
  #print(prob_prediction)

  #print("prediction: "+str(prediction.numpy()[0]) +" with probability(certainty): "+str(prob_prediction.numpy()))
  
  #if probability(certainty) is bigger than 80%
  if prob_prediction>0.8:
    return prediction.numpy()[0]
  else:
    return -1


In [ ]:
testsent="I am sad"
print(str(classify_emotion(testsent)))


4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
index2emotion={
    0:"anger", 1:"fear", 2:"joy", 3:"love", 4:"sadness", 5:"surprise"
}

<h1>Get the Poetry dataset</h1>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import spatial
from collections import Counter

import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GRU

import matplotlib.pyplot as plt

In [ ]:
poems_df = pd.read_csv("/content/drive/MyDrive/PoemDataset/kaggle_poem_dataset.csv")

In [ ]:
len(poems_df)

15652

<h1> Classify the poems by Emotion <h1>

In [ ]:
anger_set=[]
fear_set=[]
joy_set=[]
love_set=[]
sadness_set=[]
surprise_set=[]

for i in range(0,1000):
  print(i)
  #print(poems_df.iloc[i]["Author"])
  #print(poems_df.iloc[i]["Title"])
  #print(poems_df.iloc[i]["Content"])
  content=poems_df.iloc[i]["Content"]

  #Bert has max length 512
  if len(content)>512:
    content=content[0:512]

  emotion_idx=classify_emotion(content)
  if emotion_idx==0:
    anger_set.append(poems_df.iloc[i])
  elif emotion_idx==1:
    fear_set.append(poems_df.iloc[i])
  elif emotion_idx==2:
    joy_set.append(poems_df.iloc[i])
  elif emotion_idx==3:
    love_set.append(poems_df.iloc[i])
  elif emotion_idx==4:
    sadness_set.append(poems_df.iloc[i])
  elif emotion_idx==5:
    surprise_set.append(poems_df.iloc[i])
  #if emotion_idx=-1 -> the certainty of the result is lower than 80%


  #print("Classification Result: "+str(classify_emotion(content)))
  #print("##################################")

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
print(str(len(anger_set))+";"+str(len(fear_set))+";"+str(len(joy_set))+";"+str(len(love_set))+";"+str(len(sadness_set))+";"+str(len(surprise_set))+";")


121;82;203;41;199;11;


In [ ]:
def printpoemset(poemset):
  for poem in poemset:
    print("Author: "+poem["Author"])
    print("Title: "+poem["Title"])
    print(poem["Content"])
    print()

<h1>saving results </h1>

txt approach

csv approach

In [ ]:
import csv

def save_result_csv(path,poemset):
  with open(path,'w',newline='') as csvfile:
    fieldnames=['Id','Author','Title','Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    cnt=0
    for poem in poemset:
      cnt=cnt+1
      writer.writerow({"Id":cnt,"Author":poem["Author"],"Title":poem["Title"],"Content":poem["Content"]})

In [ ]:
#import csv
#with open("/content/drive/MyDrive/EmotionPoem/anger.csv",'w',newline='') as csvfile:
  #fieldnames=['Author','Title','Content']
  #writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  #writer.writeheader()
  #for poem in anger_set:
    #writer.writerow({"Author":poem["Author"],"Title":poem["Title"],"Content":poem["Content"]})

In [ ]:
save_result_csv("/content/drive/MyDrive/EmotionPoem/joy_poems.csv",joy_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/sadness_poems.csv",sadness_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/anger_poems.csv",anger_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/fear_poems.csv",fear_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/love_poems.csv",love_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/surprise_poems.csv",surprise_set)

In [ ]:
import csv

def read_from_csv(path):
  with open(path,newline='') as csvfile:
    rows=csv.DictReader(csvfile)
    row_cnt=0
    for row in rows:
      print(str(row_cnt)+":"+row["Author"]+row["Title"])
      row_cnt=row_cnt+1
  print("row count:"+str(row_cnt))
    

In [ ]:
read_from_csv("/content/drive/MyDrive/EmotionPoem/love_poems.csv")

0:Jill Alexander Essbaum4:13 am
1:Rafael CampoThe Abdominal Exam
2:Dick AllenThe Accompanist
3:Pam RehmActs of Love
4:Thomas ChattertonÆlla, a Tragical Interlude
5:William FaulknerAfter Fifty Years
6:Peter PereiraAfter
  
   The Pillow Book
  
 
 
  
   Launch Audio in a New Window
7:Henry Wadsworth LongfellowAftermath
8:Ross GayAgain
9:Second Baron Vaux of Harrowden Thomas, Lord VauxThe Aged Lover Renounceth Love
10:William BlakeAh! Sun-flower
11:John DonneAir and Angels
12:Jason GurielAlchemy
13:Carol FrostAll Summer Long
14:AnonymousAlysoun
15:Vera PavlovaAm I lovely? Of course!
16:Yehuda AmichaiThe Amen Stone
17:Esther LeeAmening
18:Robert BrowningAmong the Rocks
19:Edmund SpenserAmoretti LXVII: Like as a Huntsman
20:Rae ArmantroutAnd
21:Natalie ShaperoAnd Also with You
22:Hilda MorleyAnd I in My Bed Again
23:Edgar Allan PoeAnnabel Lee
24:Cecilia WolochAnniversary
25:Dunya MikhailAnother Planet
26:E. E. Cummings[anyone lived in a pretty how town]
 
 
  
   Launch Audio in a New Win

In [ ]:
#import csv
#with open("/content/drive/MyDrive/EmotionPoem/anger.csv",newline='') as csvfile:
#  rows=csv.DictReader(csvfile)
#  for row in rows:
#    print(row["Author"])

Naomi Shihab Nye
Charles Bukowski
Yolanda Wisher
Natalie Diaz
Ron Padgett
Jill McDonough
William E. Stafford
Bill Berkson
Myung Mi Kim
Cally Conan-Davies
Danez Smith
Donald Davie
A. E. Stallings
Margo Tamez
Fanny Howe
Keith Waldrop
Michael Earl Craig
Stephanie Burt
Anna Wickham
Bernadette Mayer
Rachel Wetzsteon
Justice Ameer
Ailbhe Darcy
Aimee Nezhukumatathil
Andrew Hudgins
Tom Sleigh
Audre Lorde
David Baker
Ha Jin
Timothy Murphy
Philip Schultz
Ernest M. Hemingway
Joy Harjo
Álvaro de Campos
Marge Piercy
Stevie Smith
Muriel Rukeyser
Sir  Thomas Wyatt
Warren Woessner
Tomas Tranströmer
Rachel Wetzsteon
David Hernandez
Christian Wiman
Alli Warren
Cornelius Eady
Kobayashi Issa
Mary Karr
Timothy Liu
C. D. Wright
James Longenbach
Aziza Barnes
Alison Pelegrin
Linda Gregg
Orlando Ricardo Menes
Stephanie Ellis Schlaifer
Michael S. Harper
Terrance Hayes
Caroline Bird
Edmund Spenser
Carolyn Kizer
Cate Marvin
Graham Foust
Cole Swensen
Brenda Hillman
Louise Glück
Alicia Ostriker
Robert Herrick
David